In [3]:
# fundemental modules
import gymnasium as gym
import matplotlib.pyplot as plt
import numpy as np
import time
import pprint
from tqdm.notebook import trange

from agent import Agent # type: ignore
from utils import load_config
from obs import observation_shape

aggregate_stats_every=100
best_score = -1000.0
score_history = []
episode_lens = []
avg_history = []
std_history = []
avg_history_100 = []



In [4]:
data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore
# pprint.pprint(env.config) # type: ignore
(obs, info), done = env.reset(), False

#observation config
proc = observation_shape(obs,info,2)
proc.reset()
input = proc.get_input()
print(input.shape)


(26,)


/home/o/Documents/thesis/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


In [5]:
agent = Agent(lr=0.00007, gamma=0.9, n_actions=1, epsilon=1.0,
                  batch_size=64, input_dims=input.shape)
print("Agent is initialized.")


Agent is initialized.


2023-12-11 16:42:16.398238: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-11 16:42:16.422163: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-12-11 16:42:16.422273: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [6]:

#TODO EDIT


# main loop
for episode in trange(500, desc='Test episodes'):

        proc.reset()
        (observation, info), done = env.reset(), False
        truncated = False
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) and (not truncated):
            
            action, action_index = agent.choose_action(observation)
            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1
            
            agent.store_transition(state=observation, action=action_index, done=done,
                            reward=reward, new_state=new_observation)
            agent.learn()

            observation = new_observation

        episode_lens.append(episode_len)

        score_history.append(episode_reward)
        avg_score = np.mean(score_history)
        avg_history.append(avg_score)
        std_score = np.std(score_history)
        std_history.append(std_score)

        avg_score_100 = np.mean(score_history[-100:])
        avg_history_100.append(avg_score_100)

        agent.tensorboard.update_stats(episode_rew = episode_reward,
                                       average_rew =avg_score,
                                       average_100_reward = avg_score_100,
                                       std_rew=std_score,
                                       epsilon=agent.epsilon,
                                       episode_len = episode_len)
        

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len,
              'avg score %.1f' % avg_score, 'avg_score_100 %.1f' %avg_score_100,'std score %.1f' % std_score)

Test episodes:   0%|          | 0/500 [00:00<?, ?it/s]

2023-12-11 16:42:17.964989: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


episode  0 score 1.7 ep len 23 avg score 1.7 avg_score_100 1.7 std score 0.0
episode  1 score 0.6 ep len 9 avg score 1.2 avg_score_100 1.2 std score 0.6
episode  2 score 0.4 ep len 7 avg score 0.9 avg_score_100 0.9 std score 0.6
episode  3 score 0.6 ep len 8 avg score 0.8 avg_score_100 0.8 std score 0.5


2023-12-11 16:42:19.646747: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f82902b5280 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-11 16:42:19.646761: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2023-12-11 16:42:19.649665: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-12-11 16:42:20.515068: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8902
2023-12-11 16:42:20.575717: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


episode  4 score 5.0 ep len 17 avg score 1.7 avg_score_100 1.7 std score 1.7
episode  5 score 2.3 ep len 16 avg score 1.8 avg_score_100 1.8 std score 1.6
episode  6 score 0.4 ep len 6 avg score 1.6 avg_score_100 1.6 std score 1.5
episode  7 score 0.7 ep len 6 avg score 1.5 avg_score_100 1.5 std score 1.5
episode  8 score 0.6 ep len 12 avg score 1.4 avg_score_100 1.4 std score 1.4
episode  9 score 3.5 ep len 15 avg score 1.6 avg_score_100 1.6 std score 1.5
episode  10 score 0.8 ep len 9 avg score 1.5 avg_score_100 1.5 std score 1.4
episode  11 score 4.6 ep len 15 avg score 1.8 avg_score_100 1.8 std score 1.6
episode  12 score 1.1 ep len 4 avg score 1.7 avg_score_100 1.7 std score 1.6
episode  13 score 0.9 ep len 3 avg score 1.6 avg_score_100 1.6 std score 1.5
episode  14 score 0.9 ep len 5 avg score 1.6 avg_score_100 1.6 std score 1.5
episode  15 score 1.0 ep len 5 avg score 1.6 avg_score_100 1.6 std score 1.4
episode  16 score 0.6 ep len 9 avg score 1.5 avg_score_100 1.5 std score 1.4


In [ ]:
n_steps = sum(episode_lens)
print(n_steps)


In [ ]:
# env = record_videos(env)
from gymnasium.wrappers import RecordVideo

data = load_config()
env = gym.make('racetrack-v0', render_mode = 'rgb_array')
env.configure(data) # type: ignore

env = RecordVideo(env, video_folder="videos", episode_trigger=lambda e: True)
env.unwrapped.set_record_video_wrapper(env)

# main loop
for episode in trange(1, desc='Test episodes'):

        proc.reset()
        (observation, info), done = env.reset(), False
        truncated = False
        proc.update_input(observation, info)
        observation = proc.get_input()

        episode_reward = 0
        episode_len = 0

        while (not done) or truncated:
            
            action, action_index = agent.get_action(observation, deterministic=True)
            new_observation, reward, done, truncated, new_info = env.step(action=[action])
            
            proc.update_input(new_observation, info)
            new_observation = proc.get_input()

            episode_reward += reward # type: ignore
            episode_len +=1

            if new_info["rewards"]["on_road_reward"] == False or episode_len >=150:
               done = True
            
            observation = new_observation

        print('episode ', episode, 'score %.1f' % episode_reward, 'ep len', episode_len)   

env.close()     